<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [0]:
# TODO - Words, words, mere words, no matter from the heart.
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical

import numpy as np
import matplotlib.pyplot as plt
import requests
import os
import textwrap

url = "https://www.gutenberg.org/files/100/100-0.txt"

In [3]:
# Preprocessing
path_to_file = tf.keras.utils.get_file('shakespeare.txt', url)

text = open(path_to_file, 'rb').read().decode(encoding='utf-8-sig')

# remove \r character
# first 900 words are headers and last 25000 words are disclaimers - remove junk
# remove all extra spaces
text = text.replace("\r", "")
text = text[900:-25000].lower()
text = " ".join(text.split())
print (f'Length of text: {len(text)} characters')

5791744/5784758 [==============================] - 4s 1us/step
Length of text: 5252797 characters


In [0]:
text_size = len(text)
vocab = sorted(set(text))
small_text_size = 100000

# create dictionary to translate characters to int and vice versa
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = {i:u for i, u in enumerate(vocab)}

text_as_int = np.array([char2idx[c] for c in text])  #encode input data

seq_length = 100

X_text = []
y_text = []

for i in range(0, small_text_size - seq_length, 1):
	in_seq = text[i:i + seq_length]
	out_char = text[i + seq_length]
	X_text.append([char2idx[char] for char in in_seq])
	y_text.append(char2idx[out_char])
    
samples = len(X_text)

In [0]:
X = np.reshape(X_text, (samples, seq_length, 1))
# normalize
X = X / float(len(vocab))
# one hot encode the output variable
y = to_categorical(y_text)

In [0]:
# hyperparameters
num_chars = len(vocab)

iteration = 1
sequence_length = 40
batch_size = round((text_size /sequence_length)+0.5) # = math.ceil
hidden_size = 500  # size of hidden layer of neurons.  
learning_rate = 1e-1


# model parameters

W_xh = np.random.randn(hidden_size, num_chars)*0.01     # weight input -> hidden. 
W_hh = np.random.randn(hidden_size, hidden_size)*0.01   # weight hidden -> hidden
W_hy = np.random.randn(num_chars, hidden_size)*0.01     # weight hidden -> output

b_h = np.zeros((hidden_size, 1)) # hidden bias
b_y = np.zeros((num_chars, 1)) # output bias

h_prev = np.zeros((hidden_size,1)) # h_(t-1)

In [0]:
def forwardprop(inputs, targets, h_prev):
        
    # Since the RNN receives the sequence, the weights are not updated during one sequence.
    xs, hs, ys, ps = {}, {}, {}, {} # dictionary
    hs[-1] = np.copy(h_prev) # Copy previous hidden state vector to -1 key value.
    loss = 0 # loss initialization
    
    for t in range(len(inputs)): # t is a "time step" and is used as a key(dic).  
        
        xs[t] = np.zeros((num_chars,1)) 
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(W_xh, xs[t]) + np.dot(W_hh, hs[t-1]) + b_h) # hidden state. 
        ys[t] = np.dot(W_hy, hs[t]) + b_y # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars. 
        
        # Softmax. -> The sum of probabilities is 1 even without the exp() function, but all of the elements are positive through the exp() function.
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss). Efficient and simple code

#         y_class = np.zeros((num_chars, 1)) 
#         y_class[targets[t]] =1
#         loss += np.sum(y_class*(-np.log(ps[t]))) # softmax (cross-entropy loss)        

    return loss, ps, hs, xs

In [0]:
def backprop(ps, inputs, hs, xs, targets):

    dWxh, dWhh, dWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy) # make all zero matrices.
    dbh, dby = np.zeros_like(b_h), np.zeros_like(b_y)
    dhnext = np.zeros_like(hs[0]) # (hidden_size,1) 

    # reversed
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t]) # shape (num_chars,1).  "dy" means "dloss/dy"
        dy[targets[t]] -= 1 # backprop into y. After taking the soft max in the input vector, subtract 1 from the value of the element corresponding to the correct label.
        dWhy += np.dot(dy, hs[t].T)
        dby += dy 
        dh = np.dot(W_hy.T, dy) + dhnext # backprop into h. 
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity #tanh'(x) = 1-tanh^2(x)
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(W_hh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]: 
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients.  
    
    return dWxh, dWhh, dWhy, dbh, dby

In [13]:
%%time

data_pointer = 0

# memory variables for Adagrad
mWxh, mWhh, mWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy)
mbh, mby = np.zeros_like(b_h), np.zeros_like(b_y) 

for i in range(iteration):
    h_prev = np.zeros((hidden_size,1)) # reset RNN memory
    data_pointer = 0 # go from start of data
    
    for b in range(batch_size):
        
        inputs = [char2idx[ch] for ch in text[data_pointer:data_pointer+sequence_length]]
        targets = [char2idx[ch] for ch in text[data_pointer+1:data_pointer+sequence_length+1]] # t+1        
            
        if (data_pointer+sequence_length+1 >= len(text) and b == batch_size-1): # processing of the last part of the input data. 
#             targets.append(char_to_int[txt_data[0]])   # When the data doesn't fit, add the first char to the back.
            targets.append(char_to_int[" "])   # When the data doesn't fit, add space(" ") to the back.


        # forward
        loss, ps, hs, xs = forwardprop(inputs, targets, h_prev)
#         print(loss)
    
        # backward
        dWxh, dWhh, dWhy, dbh, dby = backprop(ps, inputs, hs, xs, targets) 
        
        
    # perform parameter update with Adagrad
        for param, dparam, mem in zip([W_xh, W_hh, W_hy, b_h, b_y], 
                                    [dWxh, dWhh, dWhy, dbh, dby], 
                                    [mWxh, mWhh, mWhy, mbh, mby]):
            mem += dparam * dparam # elementwise
            param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update      
    
        data_pointer += sequence_length # move data pointer
        
    if i % 100 == 0:
        print ('iter %d, loss: %f' % (i, loss)) # print progress

KeyboardInterrupt: ignored

In [0]:
def predict(test_char, length):
    x = np.zeros((num_chars, 1)) 
    x[char2idx[test_char]] = 1
    ixes = []
    h = np.zeros((hidden_size,1))

    for t in range(length):
        h = np.tanh(np.dot(W_xh, x) + np.dot(W_hh, h) + b_h) 
        y = np.dot(W_hy, h) + b_y
        p = np.exp(y) / np.sum(np.exp(y)) 
        ix = np.random.choice(range(num_chars), p=p.ravel()) # ravel -> rank0
        # "ix" is a list of indexes selected according to the soft max probability.
        x = np.zeros((num_chars, 1)) # init
        x[ix] = 1 
        ixes.append(ix) # list
    txt = test_char + ''.join(idx2char[i] for i in ixes)
    print ('----\n %s \n----' % (txt, ))

In [22]:
predict('t', 50)

----
 thet hiy’gl han dow dhith dons erlrte norerwowtirom 
----


In [23]:
# Use keras instead (I don't have the compute power to get this done manually)
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

epochs = 10
batch_size = 1024

In [28]:
history = model.fit(X, y, batch_size=batch_size, epochs=epochs, callbacks=[checkpoint_callback])

Train on 99900 samples
Epoch 1/10
99900/99900 [==============================] - 1206s 12ms/sample - loss: 3.0382
Epoch 2/10
99900/99900 [==============================] - 1211s 12ms/sample - loss: 2.9637
Epoch 3/10
99900/99900 [==============================] - 1199s 12ms/sample - loss: 2.9147
Epoch 4/10
99900/99900 [==============================] - 1194s 12ms/sample - loss: 2.8138
Epoch 5/10
99900/99900 [==============================] - 1196s 12ms/sample - loss: 2.7510
Epoch 6/10
99900/99900 [==============================] - 1203s 12ms/sample - loss: 2.7128
Epoch 7/10
99900/99900 [==============================] - 1206s 12ms/sample - loss: 2.6773
Epoch 8/10
99900/99900 [==============================] - 1201s 12ms/sample - loss: 2.6487
Epoch 9/10
99900/99900 [==============================] - 1193s 12ms/sample - loss: 2.6226
Epoch 10/10
99900/99900 [==============================] - 1152s 12ms/sample - loss: 2.5933


In [29]:
start = np.random.randint(0, len(X_text)-1)
vocab_len = len(vocab)
pattern = X_text[start]

print(f"Seed: \n {''.join([idx2char[value] for value in pattern])}")
out = [idx2char[value] for value in pattern]

# generate characters
for i in range(500):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(vocab_len)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = idx2char[index]
    in_seq = [idx2char[value] for value in pattern]
    out.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

print("\nGenerated Text:\n")
print(textwrap.fill(''.join(out), 80))

Seed: 
 nceit of thine in thy soul’s thought (all naked) will bestow it: till whatsoever star that guides my

Generated Text:

nceit of thine in thy soul’s thought (all naked) will bestow it: till whatsoever
star that guides my the toe toe toe toee the toe toe toe toee the toe toe toe
toee the toe toe toe toee the toe toe toe toee the toe toe toe toee the toe toe
toe toee the toe toe toe toee the toe toe toe toee the toe toe toe toee the toe
toe toe toee the toe toe toe toee the toe toe toe toee the toe toe toe toee the
toe toe toe toee the toe toe toe toee the toe toe toe toee the toe toe toe toee
the toe toe toe toee the toe toe toe toee the toe toe toe toee the toe toe toe
toee the toe toe toe toee the toe toe toe


# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN